In [1]:
#coding=utf-8
import numpy as np
import pandas as pd
from scipy.stats import mode
import xgboost as xgb
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from sklearn.metrics import mean_squared_error,roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from matplotlib import  pyplot
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn import preprocessing  
min_max_scaler = preprocessing.MinMaxScaler()  
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.model_selection import train_test_split

llb


In [4]:
data = pd.read_csv("./willnorm/content_ext_crd_cd_lnd.csv")
print(data.head())
print(data.columns)
print(data.index)

   REPORT_ID  state_num  finance_org_num  currency_num  guarantee_type_num  \
0     8787.0        2.0              2.0           2.0                 2.0   
1     9410.0       19.0             19.0          19.0                19.0   
2    22999.0       14.0             14.0          14.0                14.0   
3    24820.0       10.0             10.0          10.0                10.0   
4    25708.0        6.0              6.0           6.0                 6.0   

   payment_state_num  cardtype_num  open_date_max  open_date_min  \
0                2.0           2.0   1.399392e+09   1.399392e+09   
1               19.0          19.0   1.471363e+09   1.409501e+09   
2               14.0          14.0   1.454342e+09   1.346256e+09   
3               10.0          10.0   1.474560e+09   1.357402e+09   
4                6.0           6.0   1.458922e+09   1.456157e+09   

   open_date_mean      ...        finance_org_more  finance_org_less  \
0    1.399392e+09      ...                     2.0

In [3]:
# 6 个类别 *2
label =[ 'state_more', 'state_less', 'finance_org_more', 'finance_org_less', 'currency_more', 'currency_less', 'guarantee_type_more',
        'guarantee_type_less','payment_state_more', 'payment_state_less', 'cardtype_more', 'cardtype_less']
#
cate = []
c = 0
size = 0
for i in range(len(label)):
    tmp = []
    tmp = pd.get_dummies(data[label[i]],prefix = label[i])
    size += tmp.shape[1]
    if c == 0:
        cate = tmp
        c = 3
        continue
    cate = pd.concat([cate,tmp],axis = 1)
cate.head()

,state_more_1.0,state_more_2.0,state_more_3.0,state_more_4.0,state_more_5.0,state_more_6.0,state_more_7.0,state_more_8.0,state_more_9.0,state_more_10.0,...,cardtype_less_53.0,cardtype_less_55.0,cardtype_less_59.0,cardtype_less_60.0,cardtype_less_74.0,cardtype_less_82.0,cardtype_less_83.0,cardtype_less_90.0,cardtype_less_102.0,cardtype_less_116.0
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
label = data.iloc[:,1:-12].columns
# print(label)
number = data.iloc[:,1:-12]
kk = min_max_scaler.fit_transform(number)  
numbernew = pd.DataFrame(kk,columns = label)
fulldata = pd.concat([data.REPORT_ID,numbernew,cate],axis = 1)
fulldata.head()

,REPORT_ID,state_num,finance_org_num,currency_num,guarantee_type_num,payment_state_num,cardtype_num,open_date_max,open_date_min,open_date_mean,...,cardtype_less_53.0,cardtype_less_55.0,cardtype_less_59.0,cardtype_less_60.0,cardtype_less_74.0,cardtype_less_82.0,cardtype_less_83.0,cardtype_less_90.0,cardtype_less_102.0,cardtype_less_116.0
0,8787.0,0.008696,0.008696,0.008696,0.008696,0.008696,0.008696,0.769071,0.868774,0.771840,...,0,0,0,0,0,0,0,0,0,0
1,9410.0,0.156522,0.156522,0.156522,0.156522,0.156522,0.156522,0.926360,0.881526,0.859021,...,0,0,0,0,0,0,0,0,0,0
2,22999.0,0.113043,0.113043,0.113043,0.113043,0.113043,0.113043,0.889162,0.801744,0.813964,...,0,0,0,0,0,0,0,0,0,0
3,24820.0,0.078261,0.078261,0.078261,0.078261,0.078261,0.078261,0.933346,0.815804,0.811901,...,0,0,0,0,0,0,0,0,0,0
4,25708.0,0.043478,0.043478,0.043478,0.043478,0.043478,0.043478,0.899169,0.940381,0.899659,...,0,0,0,0,0,0,0,0,0,0


In [8]:
#删掉一些维度
train = pd.read_csv("../input/contest_basic_train.tsv",sep='\t')
mm = pd.concat([train.REPORT_ID,train.Y],axis = 1)
ceshidata = fulldata.merge(mm,how = 'inner',left_on = 'REPORT_ID',right_on = 'REPORT_ID') 
print(ceshidata.head())
print(ceshidata.shape)

  REPORT_ID  state_num  finance_org_num  currency_num  guarantee_type_num  \
0      8787   0.008696         0.008696      0.008696            0.008696   
1      9410   0.156522         0.156522      0.156522            0.156522   
2     22999   0.113043         0.113043      0.113043            0.113043   
3     24820   0.078261         0.078261      0.078261            0.078261   
4     25708   0.043478         0.043478      0.043478            0.043478   

   payment_state_num  cardtype_num  open_date_max  open_date_min  \
0           0.008696      0.008696       0.769071       0.868774   
1           0.156522      0.156522       0.926360       0.881526   
2           0.113043      0.113043       0.889162       0.801744   
3           0.078261      0.078261       0.933346       0.815804   
4           0.043478      0.043478       0.899169       0.940381   

   open_date_mean ...  cardtype_less_55.0  cardtype_less_59.0  \
0        0.771840 ...                   0                   0  

In [9]:
ceshidata.Y.value_counts()

0    28053
1     1872
Name: Y, dtype: int64

In [8]:
# from imblearn.over_sampling import SMOTE, ADASYN
# X_resampled, y_resampled = SMOTE(kind='borderline1').fit_sample(ceshidata.iloc[:,1:-1], ceshidata.iloc[:,-1])

In [10]:

# print(X_resampled[0:10,:])
# print(X_resampled.shape)
# print(y_resampled[0:10])
# print(y_resampled.shape)
# pd.Series(y_resampled).value_counts()
# x_train,x_valid,y_train, y_valid= train_test_split(X_resampled,y_resampled,test_size = 0.3)
x_train,x_valid,y_train, y_valid= train_test_split(ceshidata.iloc[:,1:-1],ceshidata.iloc[:,-1],test_size = 0.3)
print(x_train.shape)
print(y_train.shape)
print(x_valid.shape)
print(y_valid.shape)

(20947, 579)
(20947,)
(8978, 579)
(8978,)


In [11]:
pd.Series(y_train).value_counts()

0    19626
1     1321
Name: Y, dtype: int64

In [12]:
# x_train = ceshidata.iloc[0:18000,1:-1]
# y_train = ceshidata.iloc[0:18000,-1]
# x_valid = ceshidata.iloc[18000:,1:-1]
# y_valid = ceshidata.iloc[18000:,-1]

skf = StratifiedKFold(n_splits=5)
xgb1 = XGBClassifier(
        learning_rate =0.1,
        n_estimators=1000,
        max_depth=5,
        min_child_weight=1,
        gamma=0,
        subsample=0.8,
        colsample_bytree=0.8,
        objective= 'binary:logistic',
        nthread=3,
        scale_pos_weight=1,
        StratifiedKFold = True,
        seed=27)
xgb_param = xgb1.get_xgb_params()
#构建稀疏矩阵，运行更快
xgtrain = xgb.DMatrix(x_train, label = y_train)
# xgtest =xgb.DMatrix(dtest[predictors].values)
mm = xgb.cv(xgb_param, xgtrain, num_boost_round=1000, nfold=3,
    metrics='auc', early_stopping_rounds=50)
mm

,test-auc-mean,test-auc-std,train-auc-mean,train-auc-std
0,0.601274,0.009330,0.642731,0.007235
1,0.618955,0.006983,0.669195,0.013051
2,0.622831,0.009367,0.678312,0.007087
3,0.627778,0.009446,0.686263,0.003071
4,0.629273,0.008307,0.692659,0.004974
5,0.629064,0.006678,0.700611,0.008772
6,0.627453,0.009432,0.707787,0.008530
7,0.627863,0.009673,0.709829,0.008996
8,0.630290,0.009792,0.714414,0.007882
9,0.632034,0.009190,0.720295,0.008875


In [13]:

print(x_train.shape)
print(y_train.shape)
print(x_valid.shape)
print(y_valid.shape)

(20947, 579)
(20947,)
(8978, 579)
(8978,)


In [14]:
feature = 0
for i in range(5):
    xgb2 = XGBClassifier(
            learning_rate =0.1,
            n_estimators=mm.shape[0],
            max_depth=5,
            min_child_weight=1,
            gamma=0,
            subsample=0.8,
            colsample_bytree=0.8,
            objective= 'binary:logistic',
            nthread=4,
            scale_pos_weight=1,   #15:1
            StratifiedKFold = True,
            seed=27)
    xgb2.fit(x_train,y_train)
    feature += xgb2.feature_importances_
    print(i)
#     pred = xgb2.predict_proba(x_valid)[:,1]
#     print(pred)
#     auc = roc_auc_score(y_valid, pred)  
#     auc
feature = feature/5

0
1
2
3
4


In [16]:
feature

array([0.00544323, 0.0007776 , 0.        , 0.        , 0.        ,
       0.        , 0.02799378, 0.05598756, 0.04743391, 0.00233281,
       0.        , 0.01399689, 0.00777605, 0.01088647, 0.0007776 ,
       0.        , 0.00777605, 0.00699844, 0.02721617, 0.00699844,
       0.        , 0.01166407, 0.00622084, 0.02177294, 0.00233281,
       0.        , 0.01166407, 0.01399689, 0.01788491, 0.01321928,
       0.        , 0.02177294, 0.03421462, 0.01088647, 0.00699844,
       0.        , 0.02255054, 0.02177294, 0.03343701, 0.        ,
       0.        , 0.02799378, 0.00466563, 0.01866252, 0.00311042,
       0.        , 0.01010886, 0.01010886, 0.0163297 , 0.00233281,
       0.        , 0.02721617, 0.01088647, 0.01788491, 0.0007776 ,
       0.        , 0.01866252, 0.02643857, 0.02566097, 0.0007776 ,
       0.        , 0.02099533, 0.01010886, 0.02332815, 0.00233281,
       0.        , 0.02332815, 0.00544323, 0.03032659, 0.00544323,
       0.        , 0.02177294, 0.02877138, 0.02877138, 0.00155

In [18]:
print(x_train.columns)
print(feature[feature>0.0000008])
x_train.columns[feature>0.0000008]

Index(['state_num', 'finance_org_num', 'currency_num', 'guarantee_type_num',
       'payment_state_num', 'cardtype_num', 'open_date_max', 'open_date_min',
       'open_date_mean', 'open_date_zongshu',
       ...
       'cardtype_less_53.0', 'cardtype_less_55.0', 'cardtype_less_59.0',
       'cardtype_less_60.0', 'cardtype_less_74.0', 'cardtype_less_82.0',
       'cardtype_less_83.0', 'cardtype_less_90.0', 'cardtype_less_102.0',
       'cardtype_less_116.0'],
      dtype='object', length=579)
[0.00544323 0.0007776  0.02799378 0.05598756 0.04743391 0.00233281
 0.01399689 0.00777605 0.01088647 0.0007776  0.00777605 0.00699844
 0.02721617 0.00699844 0.01166407 0.00622084 0.02177294 0.00233281
 0.01166407 0.01399689 0.01788491 0.01321928 0.02177294 0.03421462
 0.01088647 0.00699844 0.02255054 0.02177294 0.03343701 0.02799378
 0.00466563 0.01866252 0.00311042 0.01010886 0.01010886 0.0163297
 0.00233281 0.02721617 0.01088647 0.01788491 0.0007776  0.01866252
 0.02643857 0.02566097 0.0007776  0

Index(['state_num', 'finance_org_num', 'open_date_max', 'open_date_min',
       'open_date_mean', 'open_date_zongshu', 'credit_limit_amount_max',
       'credit_limit_amount_min', 'credit_limit_amount_mean',
       'credit_limit_amount_zongshu',
       ...
       'guarantee_type_less_14.0', 'guarantee_type_less_18.0',
       'payment_state_more_1.0', 'payment_state_more_2.0',
       'payment_state_more_4.0', 'payment_state_more_6.0',
       'payment_state_more_10.0', 'cardtype_more_2.0', 'cardtype_more_4.0',
       'cardtype_more_18.0'],
      dtype='object', length=117)

In [19]:
fulldata.head()
fulldataok = []
fulldataok = fulldata[x_train.columns[feature>0.0000008]]  #一定要小心是谁的ID
fulldataok = pd.concat([fulldata.REPORT_ID,fulldataok],axis = 1)
fulldataok.head()

,REPORT_ID,state_num,finance_org_num,open_date_max,open_date_min,open_date_mean,open_date_zongshu,credit_limit_amount_max,credit_limit_amount_min,credit_limit_amount_mean,...,guarantee_type_less_14.0,guarantee_type_less_18.0,payment_state_more_1.0,payment_state_more_2.0,payment_state_more_4.0,payment_state_more_6.0,payment_state_more_10.0,cardtype_more_2.0,cardtype_more_4.0,cardtype_more_18.0
0,8787.0,0.008696,0.008696,0.769071,0.868774,0.771840,0.026316,0.020446,0.044265,0.044454,...,0,0,1,0,0,0,0,1,0,0
1,9410.0,0.156522,0.156522,0.926360,0.881526,0.859021,0.026316,0.016357,0.000000,0.018484,...,0,0,0,0,0,0,0,0,0,0
2,22999.0,0.113043,0.113043,0.889162,0.801744,0.813964,0.026316,0.073605,0.000000,0.025514,...,1,0,0,0,1,0,0,0,0,0
3,24820.0,0.078261,0.078261,0.933346,0.815804,0.811901,0.026316,0.047707,0.017857,0.049505,...,0,0,1,0,0,0,0,0,0,0
4,25708.0,0.043478,0.043478,0.899169,0.940381,0.899659,0.026316,0.013915,0.011905,0.020937,...,0,0,1,0,0,0,0,0,0,0


In [20]:
fulldataok.to_csv("../add/content_ext_crd_cd_lndok.csv",index = False)

In [21]:
fulldataok.shape

(39850, 118)